# E6. 프로젝트: 멋진 작사가 만들기

## 루브릭
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
* 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
* 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?
* 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

## Step 1. 데이터 다운로드
먼저 아래 링크에서 Song Lyrics 데이터를 다운로드해 주세요! 저장된 파일을 압축 해제한 후, 모든 txt 파일을 lyrics 폴더를 만들어 그 속에 저장해주세요!

* `wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip`  
* `unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics`      # lyrics 폴더에 압축풀기

## Step 2. 데이터 읽어오기
glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요.   
glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!

import glob                # 파일 처리
import os                  # 경로 처리


In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

# 파일들의 리스트를 뽑을 때 glob.glob() 사용
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


#### `.extend()` 함수는 `.append()` 함수와 차이가 있다.
* **`list.append(x)`** 는 **리스트 끝에 x 1개를 그대로 넣는다.**
* **`list.extend(iterable)`** 는 **리스트 끝에 가장 바깥쪽 iterable의 모든 항목을 넣는다**

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[Hook]
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face
I've been down so long, it look like up to me
They look up to me
I got fake people showin' fake love to me
Straight up to my face, straight up to my face [Verse 1]
Somethin' ain't right when we talkin'


## Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래가사 작사하기에 어울리지 않을수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기를 권합니다.

### Step 3-1. 문장 전처리
* **( ) 포함 내용 삭제**
```
remove_text = 'asdf(asdf)'
print(re.sub(r'\([^)]*\)', '', remove_text))
```
* **< > 포함 내용 삭제**
```
remove_text = 'asdf<asdf>'
re.sub(r'\<[^)]*\>', '', remove_text)   
```
* **[ ] 포함 내용 삭제**
```
remove_text = 'asdf[asdf]'
re.sub(r'\[[^)]*\]', '', remove_text) 
```

In [4]:
# 문장 전처리를 위한 정제 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()                          # 소문자로 바꾸고 양쪽 공백을 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)           # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z0-9?.!,¿]+", " ", sentence)      # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'                  # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


* **정규 표현식 이용한 문자열 바꾸기**
  + **`re.sub('패턴', '바꿀문자열', '문자열', 바꿀횟수)`**
   
   
* **이스케이프 \**
  + 이스케이프 문자 \는 메타문자를 일반 리터럴 문자로 취급하게끔 해 준다.
    - 예를 들어 여는 괄호 `[`는 메타 문자지만,   
    - 이스케이프 문자 \와 `[`를 같이 처리하면 리터럴 문자인 일반 대괄호 문자 `[`와 매칭될 수 있게 된다.
    
    
* **[ ] 대괄호**:여러 문자 중 하나와 일치
  + 대괄호 `[ 와 ]` 사이에 원하는 문자를 여러 개 넣으면, 문자열이 넣은 문자 중 하나와 일치하면 매칭이 이루어진다.


* **. 마침표**: 모든 문자와 일치
  + 개행문자를 제외한 모든 문자와 일치하는 경우


* **문자 집합: \w \W, \d \D, \s \S, \b \B**
  + \w, \W: 단어 문자, 비 단어 문자  
  + \d, \D: 숫자 문자, 비 숫자 문자  
  + \s, \S: 공백 문자, 비 공백 문자
  + \b, \B: 단어 경계, 비 단어 경계
    - 한 가지 주의할 점으로는 \b나 \B를 사용하기 위해서는 정규표현식 앞에 r prefix를 붙여줘야 한다는 것이다.


* **옵션: r prefix**
  + 원래 r prefix란 이스케이프 문자 \를 이스케이프 처리 문자가 아닌 일반 리터럴 문자로 인식하게끔 하는 역할을 한다.
    - 예를 들어, 리터럴 문자열 r”\n”은 \와 소문자 n 2개의 문자로 구성된다.  
  
  
* **문자열 전체 또는 행의 시작이나 끝의 대상을 대조 :** ^, $, \A, \Z
  + \A는 문자열 시작을, \Z는 문자열 끝과 일치된다.    
  
  * ^와 $는 기본적으로 행 시작과 행 끝에 일치된다.  
  
  * ^와 $는 일반적으로 \A와 \Z 앵커와 효과가 같다.




In [5]:
# 정규표현식을 이용한 corpus 생성
corpus = []

for sentence in raw_corpus:
    sentence = re.sub(r'\[[^)]*\]', '', sentence)           # 대괄호 [ ]기호 포함하여 안의 내용 삭제 
    sentence = re.sub(r'\([^)]*\)', '', sentence)           # 소괄호 ( )기호 포함하여 안의 내용 삭제 
    
    if len(sentence) == 0: continue                         # 길이가 0인 문장 제외
    if len(sentence.split()) > 13 : continue                # 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기
    
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>']

### Step 3-2. 토큰화 및 백터화

In [6]:
# 케라스를 사용한 단어 수준의 원-핫 인코딩
def tokenize(corpus):
    # 텍스트를 단어 기반으로 토큰화 ( corpus를 텐서로 변환 )
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=30000,   # 단어 빈도에 따른 사용할 단어 개수의 최대값 , 단어장의 크기는 12,000 이상으로 설정
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    # 빈도수를 기준으로 단어 사전 구축
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 문자열을 인덱스의 리스트로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 pad_sequences() 메서드 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print('데이터 개수 :', len(tensor))
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

데이터 개수 : 161317
[[  2   5  90 ...  10  12   3]
 [  2  41 132 ...   0   0   0]
 [  2   5  39 ...   0   0   0]
 ...
 [  2 152  49 ...   0   0   0]
 [  2 152   4 ...   0   0   0]
 [  2 152   4 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fc4dc42e2d0>


* **tf.keras.preprocessing.text.Tokenizer()**
    + 이 클래스는 텍스트 코퍼스를 벡터화 할 수 있음
    + 각 텍스트를 정수 시퀀스 또는 이진수인 벡터(0과 1)로 변환함
* 데이터 개수 : 166218 (if len(sentence.split()) > 15 : continue )
* 데이터 개수 : 161317 (if len(sentence.split()) > 13 : continue )


In [7]:
# 생성된 텐서 데이터를 3번째 행, 열 전체 출력
print(tensor[:3, :])

[[   2    5   90  103   58   31  164    4   11  132   24   29   10   12
     3]
 [   2   41  132   29   10   12    3    0    0    0    0    0    0    0
     0]
 [   2    5   39  815  175 2524  815   38   10   12    3    0    0    0
     0]]


시퀀스 길이를 15개로 맞추고 15개 보다 짧은 것은 뒤에 패딩을 추가함

In [8]:
# 텐서 데이터의 숫자는 tokenizer에 구축된 단어 사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰 <end>를 잘라내서 소스 문장을 생성합니다. 
tgt_input = tensor[:, 1:]    # tensor에서 맨 앞 토큰 <start>를 잘라내서 타겟 문장을 생성합니다.

# corpus 내의 첫번째 문장에 대해 생성된 소스와 타겟 문장을 확인
print(src_input[0])
print(tgt_input[0])

[  2   5  90 103  58  31 164   4  11 132  24  29  10  12]
[  5  90 103  58  31 164   4  11 132  24  29  10  12   3]


In [10]:
print("데이터 크기:", len(src_input))

데이터 크기: 161317


## Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다.     
단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!     
만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요!     
만약 학습데이터 갯수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시한번 검토해 보시기를 권합니다.

### Step 4-1.  데이터셋 분리

In [11]:
from sklearn.model_selection import train_test_split


# train / test 데이터셋 분리
train_x, test_x, train_y, test_y = train_test_split(src_input,
                                                    tgt_input,
                                                    test_size=0.2,
                                                    shuffle=True)
train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                      train_y,
                                                      test_size=0.2,
                                                      shuffle=True)

print('train_data :', train_x.shape)
print('valid_data :', valid_x.shape)
print('test_data :', test_x.shape)

train_data : (103242, 14)
valid_data : (25811, 14)
test_data : (32264, 14)


In [12]:
# # 판다스 데이터프레임으로부터 tf.data 데이터셋을 만들기 위한 함수
# def df_to_dataset(dataframe, shuffle=True, batch_size=32):
#   dataframe = dataframe.copy()
#   labels = dataframe.pop('target')
#   ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
#   if shuffle:
#     ds = ds.shuffle(buffer_size=len(dataframe))
#   ds = ds.batch(batch_size)
#   return ds

In [13]:
# batch_size = 5 # 예제를 위해 작은 배치 크기를 사용합니다.
# train_ds = df_to_dataset(train, batch_size=batch_size)
# val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
# test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [14]:
# for feature_batch, label_batch in train_ds.take(1):
#   print('전체 특성:', list(feature_batch.keys()))
#   print('나이 특성의 배치:', feature_batch['age'])
#   print('타깃의 배치:', label_batch )

### Step 4-2.  데이터셋 객체 생성

In [15]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 30,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 30,001개

# corpus 텐서를 tf.data.Dataset객체로 변환
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(BUFFER_SIZE)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)

valid_ds = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
valid_ds = valid_ds.batch(BATCH_SIZE, drop_remainder=True)

test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True)

train_ds

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!
데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

In [16]:
#  tf.keras.Model을 Subclassing하는 방식으로 만들 것
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        # 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024

# 모델 객체 생성
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
# model에 데이터를 아주 조금 태워  model.build()가 자동으로 호출
for src_sample, tgt_sample in train_ds.take(1): break
    
model(src_sample).shape

TensorShape([256, 14, 30001])

In [18]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  7680256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  30751025  
Total params: 52,070,961
Trainable params: 52,070,961
Non-trainable params: 0
_________________________________________________________________


### Step 5-1. 모델 학습

In [19]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                     reduction='none')

model.compile(loss=loss, optimizer=optimizer)

history_lyrics = model.fit(train_ds, 
                           epochs=10,
                           validation_data=valid_ds)

Epoch 1/10
403/403 [==============================] - 95s 235ms/step - loss: 3.9214 - val_loss: 3.4161
Epoch 2/10
403/403 [==============================] - 96s 237ms/step - loss: 3.2918 - val_loss: 3.2410
Epoch 3/10
403/403 [==============================] - 96s 238ms/step - loss: 3.1265 - val_loss: 3.1428
Epoch 4/10
403/403 [==============================] - 96s 239ms/step - loss: 3.0106 - val_loss: 3.0745
Epoch 5/10
403/403 [==============================] - 96s 239ms/step - loss: 2.9141 - val_loss: 3.0249
Epoch 6/10
403/403 [==============================] - 96s 239ms/step - loss: 2.8277 - val_loss: 2.9796
Epoch 7/10
403/403 [==============================] - 96s 239ms/step - loss: 2.7470 - val_loss: 2.9454
Epoch 8/10
403/403 [==============================] - 97s 240ms/step - loss: 2.6699 - val_loss: 2.9132
Epoch 9/10
403/403 [==============================] - 97s 241ms/step - loss: 2.5956 - val_loss: 2.8868
Epoch 10/10
403/403 [==============================] - 97s 240ms/step - l

In [20]:
# 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하는 함수

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
        tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [25]:
#  문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> I like you")

'<start> i like you , i m not gonna be <end> '

## 회고

1. glob.glob() : glob는 파일들의 리스트를 뽑을 때 사용하는데, 파일의 경로명을 이용해서 입맛대로 요리할 수 있답니다.
2. 정규표현식 : []내용 제거, ()내용 제거
3. fit_on_text() : 문자 데이터를 입력받아서 리스트의 형태로 변환
4. Tokenizer는 문장으로부터 단어를 토큰화하고 숫자에 대응시키는 딕셔너리를 사용할 수 있도록 합니다.
5. texts_to_sequences() 메서드는 텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환합니다.
6. pad_sequences 함수에 이 시퀀스를 입력하면 숫자 0을 이용해서 같은 길이의 시퀀스로 변환합니다.